In [1]:
import neonutilities as nu

In [2]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import glob
import re
import shutil
import laspy
from shapely import geometry

In [ ]:
## Download lidar tiles over plots at each site then subset lidar point cloud for points
# within polygon

In [3]:
def convert_laz_to_las(in_laz, out_las):
    las = laspy.read(in_laz)
    las = laspy.convert(las)
    las.write(out_las)    

In [4]:
# ---------- Set up workspace and variables ---------- #
product = "DP1.30003.001"
baseplots_fdir = glob.glob(os.path.join("/data/shared/src/arojas/NEON/data/raw/spatial/baseplots","*"))
outdir = "/data/shared/src/arojas/NEON/data/baseplots-lidar"


# for each site
for sitepoly_fdir in baseplots_fdir:
    # Set up workspace, read i polygons
    print(os.path.basename(sitepoly_fdir)[:4])
    siteid = os.path.basename(sitepoly_fdir)[:4]
    site_odir = os.path.join(outdir,siteid)
    if not os.path.isdir(site_odir):
        os.mkdir(site_odir)
    # lets read in plot data!
    sitepoly_fp = glob.glob(os.path.join(sitepoly_fdir, "*.shp"))[0]
    poly_gdf = gpd.read_file(sitepoly_fp)
    
    # list available urls for lidar! (all years available)
    years_list = nu.api.list_available_urls(product,siteid)
    years_list = [os.path.basename(yr)[:-3] for yr in years_list]
    
    # Loop through each year of lidar data, then each plot, and clip
    for yr in years_list:
        # set up year outdir
        siteyr_odir = os.path.join(site_odir,yr)
        if not os.path.isdir(siteyr_odir):
            os.mkdir(siteyr_odir)
        
        # Download las tiles
        nu.api.download_aop_files(product,siteid,yr,
                                  download_folder='./tmp_lidar',
                                  poly=sitepoly_fp)
        # Convert .laz to .las
        for (dirpath, dirnames, filenames) in os.walk('./tmp_lidar'):
            for inFile in filenames:
                if inFile.endswith('.laz'):	
                    in_laz = os.path.join(dirpath,inFile)
                    out_las = in_laz.replace('laz', 'las') 
                    # Convert to las
                    convert_laz_to_las(in_laz, out_las)
        las_fpaths = glob.glob(os.path.join('./tmp_lidar', "*.las"))
        
        # Now go through each .las file and filter for plot data!
        for fpath in las_fpaths:
            # Get file coords
            try:
                coords = re.findall("[0-9]{6}_[0-9]{7}", fpath)[0].split("_")
            except:
                # Not a file with utm coords
                print("Cant find coords!")
                continue
            # Get lower left coords
            x_coord_left = int(coords[0])
            y_coord_bottom = int(coords[1])
            # Get upper right coords
            x_coord_right = x_coord_left + 1000
            y_coord_top = y_coord_bottom + 1000
            # create a shapely polygon
            poly = geometry.Polygon([[x_coord_left, y_coord_bottom], [x_coord_left, y_coord_top],
                                    [x_coord_right,y_coord_top], [x_coord_right, y_coord_bottom]])

            # Read in file, clip for polys!
            las_src = laspy.read(fpath)

            for i in range(len(poly_gdf)):
                # Check for intersection!
                plot_poly = poly_gdf.loc[[i]]
                if plot_poly.intersects(poly).any():
                    # Lets clip!
                    minx, miny, maxx, maxy = plot_poly.bounds.values[0]
                    minx = minx-3
                    miny = miny-3
                    maxx = maxx+3
                    maxy = maxy+3

                    las_dst = laspy.create(point_format=las_src.header.point_format,
                                            file_version=las_src.header.version)
                    # Filter for points!
                    xbool = (las_src.x>=minx) & (las_src.x<=maxx)
                    ybool = (las_src.y>=miny) & (las_src.y<=maxy)
                    las_dst.points = las_src.points[xbool & ybool]
                    plotid = plot_poly['plotID'].values[0]
                    outfp = os.path.join(siteyr_odir,f"{plotid}_lidar_{yr}.las")
                    las_dst.write(outfp)
        # remove tmp_lidar directory
        shutil.rmtree("./tmp_lidar")

ABBY
['https://data.neonscience.org/api/v0/data/DP1.30003.001/ABBY/2017-06']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/ABBY/2018-07']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/ABBY/2019-07']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/ABBY/2021-07']
BARR
['https://data.neonscience.org/api/v0/data/DP1.30003.001/BARR/2017-07']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/BARR/2018-07']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/BARR/2019-07']
BART
['https://data.neonscience.org/api/v0/data/DP1.30003.001/BART/2014-06']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/BART/2016-08']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/BART/2017-08']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/BART/2018-08']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/BART/2019-08']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/BART/2022-08']
BLAN
['https://data.neonscience.org/api/v0/data/D

['https://data.neonscience.org/api/v0/data/DP1.30003.001/MOAB/2020-07']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/MOAB/2021-04']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/MOAB/2022-04']
NIWO
['https://data.neonscience.org/api/v0/data/DP1.30003.001/NIWO/2017-09']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/NIWO/2018-08']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/NIWO/2019-08']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/NIWO/2020-08']
NOGP
['https://data.neonscience.org/api/v0/data/DP1.30003.001/NOGP/2016-07']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/NOGP/2017-06']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/NOGP/2019-07']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/NOGP/2020-06']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/NOGP/2021-06']
OAES
['https://data.neonscience.org/api/v0/data/DP1.30003.001/OAES/2016-04']
['https://data.neonscience.org/api/v0/data/DP1.30

['https://data.neonscience.org/api/v0/data/DP1.30003.001/YELL/2019-07']
['https://data.neonscience.org/api/v0/data/DP1.30003.001/YELL/2020-07']
